**Setup**

In this case SQLite does not support the CREATE EXTENSION statement like PostgreSQL does. The CROSSTAB function you are trying to use is specific to PostgreSQL's "tablefunc" extension.

`CREATE EXTENSION IF NOT EXISTS tablefunc;`

To use CROSSTAB, you would need to switch to using PostgreSQL as your database engine instead of SQLite. Here's how you can perform the same task using PostgreSQL:

In [1]:
# Library
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Define the database connection parameters
db_params = {
    'host': 'localhost',
    'database': 'summer',
    'user': 'postgres',
    'password': 'admin',
    'port': '5432'  # PostgreSQL default port
}

# Connect to the 'soccer' database
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

**A basic pivot**

You have the following table of Pole Vault gold medalist countries by gender in 2008 and 2012.

```
| Gender | Year | Country |
|--------|------|---------|
| Men    | 2008 | AUS     |
| Men    | 2012 | FRA     |
| Women  | 2008 | RUS     |
| Women  | 2012 | USA     |

```

Pivot it by `Year` to get the following reshaped, cleaner table.

```
| Gender | 2008 | 2012 |
|--------|------|------|
| Men    | AUS  | FRA  |
| Women  | RUS  | USA  |

```

**Instructions**

- Create the correct extension.
- Fill in the column names of the pivoted table.

In [3]:
query = """
SELECT * FROM CROSSTAB($$
  SELECT
    Gender, Year, Country
  FROM summer
  WHERE
    Year IN (2008, 2012)
    AND Medal = 'Gold'
    AND Event = 'Pole Vault'
  ORDER By Gender ASC, Year ASC;
-- Fill in the correct column names for the pivoted table
$$) AS ct (Gender TEXT,
           "2008" TEXT,
           "2012" TEXT)

ORDER BY Gender ASC;
"""
result = pd.read_sql_query(query, engine)
result

,gender,2008,2012
0,Men,AUS,FRA
1,Women,RUS,USA


**Pivoting with ranking**

You want to produce an easy scannable table of the rankings of the three most populous EU countries by how many gold medals they've earned in the 2004 through 2012 Olympic games. The table needs to be in this format:

```
| Country | 2004 | 2008 | 2012 |
|---------|------|------|------|
| FRA     | ...  | ...  | ...  |
| GBR     | ...  | ...  | ...  |
| GER     | ...  | ...  | ...  |

```

You'll need to count the gold medals each country has earned, produce the ranks of each country by medals earned, then pivot the table to this shape.

**Instructions**

- Count the gold medals that France (`FRA`), the UK (`GBR`), and Germany (`GER`) have earned per country and year.

In [4]:
query = """
-- Count the gold medals per country and year
SELECT
  Country,
  Year,
  COUNT(*) AS Awards
FROM summer
WHERE
  Country IN ('FRA', 'GBR', 'GER')
  AND Year IN (2004, 2008, 2012)
  AND Medal = 'Gold'
GROUP BY Country, Year
ORDER BY Country ASC, Year ASC
"""
result = pd.read_sql_query(query, engine)
result

,country,year,awards
0,FRA,2004,21
1,FRA,2008,25
2,FRA,2012,30
3,GBR,2004,17
4,GBR,2008,31
5,GBR,2012,48
6,GER,2004,41
7,GER,2008,42
8,GER,2012,45


- Select the country and year columns, then rank the three countries by how many gold medals they earned per year.

In [5]:
query = """
WITH Country_Awards AS (
  SELECT
    Country,
    Year,
    COUNT(*) AS Awards
  FROM summer
  WHERE
    Country IN ('FRA', 'GBR', 'GER')
    AND Year IN (2004, 2008, 2012)
    AND Medal = 'Gold'
  GROUP BY Country, Year)

SELECT
  Country,
  Year,
  -- Rank by gold medals earned per year
  RANK() OVER
    (PARTITION BY Year
     ORDER BY Awards DESC) :: INTEGER AS rank
FROM Country_Awards
ORDER BY Country ASC, Year ASC;
"""
result = pd.read_sql_query(query, engine)
result

,country,year,rank
0,FRA,2004,2
1,FRA,2008,3
2,FRA,2012,3
3,GBR,2004,3
4,GBR,2008,2
5,GBR,2012,1
6,GER,2004,1
7,GER,2008,1
8,GER,2012,2


- Pivot the query's results by Year by filling in the new table's correct column names.

In [9]:
query = """
SELECT * FROM CROSSTAB($$
  WITH Country_Awards AS (
    SELECT
      Country,
      Year,
      COUNT(*) AS Awards
    FROM Summer
    WHERE
      Country IN ('FRA', 'GBR', 'GER')
      AND Year IN (2004, 2008, 2012)
      AND Medal = 'Gold'
    GROUP BY Country, Year)

  SELECT
    Country,
    Year,
    RANK() OVER
      (PARTITION BY Year
       ORDER BY Awards DESC) :: INTEGER AS rank
  FROM Country_Awards
  ORDER BY Country ASC, Year ASC;
-- Fill in the correct column names for the pivoted table
$$) AS ct (Country TEXT,
           "2004" INTEGER,
           "2008" INTEGER,
           "2012" INTEGER)

Order by Country ASC;
"""
result = pd.read_sql_query(query, engine)
result

,country,2004,2008,2012
0,FRA,2,3,3
1,GBR,3,2,1
2,GER,1,1,2


**Country-level subtotals**

You want to look at three Scandinavian countries' earned gold medals per country and gender in the year 2004. You're also interested in `Country`-level subtotals to get the total medals earned for each country, but `Gender`-level subtotals don't make much sense in this case, so disregard them.

**Instructions**

- Count the gold medals awarded per country and gender.
- Generate `Country`-level gold award counts.

In [10]:
query = """
-- Count the gold medals per country and gender
SELECT
  Country,
  Gender,
  COUNT(*) AS Gold_Awards
FROM summer
WHERE
  Year = 2004
  AND Medal = 'Gold'
  AND Country IN ('DEN', 'NOR', 'SWE')
-- Generate Country-level subtotals
GROUP BY Country, ROLLUP(Gender)
ORDER BY Country ASC, Gender ASC;
"""
result = pd.read_sql_query(query, engine)
result

,country,gender,gold_awards
0,DEN,Men,4
1,DEN,Women,15
2,DEN,None,19
3,NOR,Men,3
4,NOR,Women,2
5,NOR,None,5
6,SWE,Men,4
7,SWE,Women,1
8,SWE,None,5


**All group-level subtotals**

You want to break down all medals awarded to Russia in the 2012 Olympic games per gender and medal type. Since the medals all belong to one country, Russia, it makes sense to generate all possible subtotals (`Gender`- and `Medal`-level subtotals), as well as a grand total.

Generate a breakdown of the medals awarded to Russia per country and medal type, including all group-level subtotals and a grand total.

**Instructions**

- Count the medals awarded per gender and medal type.
- Generate all possible group-level counts (per gender and medal type subtotals and the grand total).

In [12]:
query = """
-- Count the medals per gender and medal type
SELECT
  Gender,
  Medal,
  COUNT(*) AS Awards
FROM summer
WHERE
  Year = 2012
  AND Country = 'RUS'
-- Get all possible group-level subtotals
GROUP BY CUBE(Gender, Medal)
ORDER BY Gender ASC, Medal ASC;
"""
result = pd.read_sql_query(query, engine)
result

,gender,medal,awards
0,Men,Bronze,34
1,Men,Gold,23
2,Men,Silver,7
3,Men,None,64
4,Women,Bronze,17
5,Women,Gold,24
6,Women,Silver,25
7,Women,None,66
8,None,Bronze,51
9,None,Gold,47


**Cleaning up results**

Returning to the breakdown of Scandinavian awards you previously made, you want to clean up the results by replacing the `null`s with meaningful text.

**Instructions**

- Turn the `null`s in the `Country` column to `All countries`, and the `null`s in the `Gender` column to `All genders`.

In [13]:
query = """
SELECT
  -- Replace the nulls in the columns with meaningful text
  COALESCE(Country, 'All countries') AS Country,
  COALESCE(Gender, 'All genders') AS Gender,
  COUNT(*) AS Awards
FROM Summer
WHERE
  Year = 2004
  AND Medal = 'Gold'
  AND Country IN ('DEN', 'NOR', 'SWE')
GROUP BY ROLLUP(Country, Gender)
ORDER BY Country ASC, Gender ASC;
"""
result = pd.read_sql_query(query, engine)
result

,country,gender,awards
0,All countries,All genders,29
1,DEN,All genders,19
2,DEN,Men,4
3,DEN,Women,15
4,NOR,All genders,5
5,NOR,Men,3
6,NOR,Women,2
7,SWE,All genders,5
8,SWE,Men,4
9,SWE,Women,1


**Summarizing results**

After ranking each country in the 2000 Olympics by gold medals awarded, you want to return the top 3 countries in one row, as a comma-separated string. In other words, turn this:

```
| Country | Rank |
|---------|------|
| USA     | 1    |
| RUS     | 2    |
| AUS     | 3    |
| ...     | ...  |

```

into this:

```
USA, RUS, AUS

```

**Instructions**

- Rank countries by the medals they've been awarded.

In [14]:
query = """
WITH Country_Medals AS (
  SELECT
    Country,
    COUNT(*) AS Medals
  FROM Summer
  WHERE Year = 2000
    AND Medal = 'Gold'
  GROUP BY Country)

  SELECT
    Country,
    -- Rank countries by the medals awarded
    RANK() OVER (ORDER BY Medals DESC) AS Rank
  FROM Country_Medals
  ORDER BY Rank ASC;
"""
result = pd.read_sql_query(query, engine)
result

,country,rank
0,USA,1
1,RUS,2
2,AUS,3
3,CHN,4
4,GER,5
5,ROU,6
6,NED,6
7,HUN,8
8,ITA,9
9,CUB,9


- Return the top 3 countries by medals awarded as one comma-separated string.

In [15]:
query = """
WITH Country_Medals AS (
  SELECT
    Country,
    COUNT(*) AS Medals
  FROM Summer
  WHERE Year = 2000
    AND Medal = 'Gold'
  GROUP BY Country),

  Country_Ranks AS (
  SELECT
    Country,
    RANK() OVER (ORDER BY Medals DESC) AS Rank
  FROM Country_Medals
  ORDER BY Rank ASC)

-- Compress the countries column
SELECT STRING_AGG(Country, ', ')
FROM Country_Ranks
-- Select only the top three ranks
WHERE Rank <= 3;
"""
result = pd.read_sql_query(query, engine)
result

,string_agg
0,"USA, RUS, AUS"
